<a href="https://colab.research.google.com/github/teamgaon/SANUP/blob/main/220317_ji_target2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gluonnlp
# !pip install mxnet
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 344 kB 7.8 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595738 sha256=49ff7118d7b3db2daad073cec1e1dfbd30fb1e69b3435951003b3cb243a0d3b4
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-a3da9dl6/kobert-tokenizer_cef65847fc814eaa876fad0c46d8ced8
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-a3da9dl6/kobert-tokenizer_cef65847fc814eaa876fad0c46d8ced8
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=b0b7b7d24d27a7fa9d7c7f98c73d1be5be8abcb0f31ddadb3d53be5d0d0985b5
  Stored in directory: /tmp/pip-ephem-wheel-cache-wuz5qf55/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer
     |████████████████████████████████

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
from sklearn.model_selection import train_test_split

In [118]:
train = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')

In [119]:
train

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육
...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스


In [120]:
train['digit_1'].value_counts()

G    246472
I    187425
C    105192
S    100396
H     98038
P     46610
L     40140
Q     36087
F     35050
R     29751
M     28434
N     17701
J     10862
K     10378
O      2965
E      2255
A      1064
D       756
B       424
Name: digit_1, dtype: int64

In [121]:
train.isnull().sum()

AI_id            0
digit_1          0
digit_2          0
digit_3          0
text_obj     16677
text_mthd    43619
text_deal    67652
dtype: int64

In [122]:
train = train.fillna(" ")

In [123]:
train.isnull().sum()

AI_id        0
digit_1      0
digit_2      0
digit_3      0
text_obj     0
text_mthd    0
text_deal    0
dtype: int64

In [131]:
df = train[train['digit_1']=="C"].copy().reset_index(drop=True)

In [132]:
df['target'] = df['digit_2']
df["text"] = df["text_obj"]+ " " + df["text_mthd"] + " " + df["text_deal"]

In [133]:
df['target'].nunique()

25

In [134]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,target,text
0,id_0000006,C,29,291,철,절삭.용접,카프라배관자재,29,철 절삭.용접 카프라배관자재
1,id_0000008,C,10,107,쌀을 가지고,가공하여,떡제조,10,쌀을 가지고 가공하여 떡제조
2,id_0000011,C,10,102,멸치,"입고, 가공",,10,"멸치 입고, 가공"
3,id_0000021,C,10,107,쌀을가지고,가루로 분쇄하여 떡을 쪄서,백설기 꿀떡 절편판매,10,쌀을가지고 가루로 분쇄하여 떡을 쪄서 백설기 꿀떡 절편판매
4,id_0000027,C,22,221,고무,절단가공,"가스켓,다이아후레임",22,"고무 절단가공 가스켓,다이아후레임"
...,...,...,...,...,...,...,...,...,...
105187,id_0999971,C,28,281,"트랜스,다이오드,칩",조립,전원공급장치(파워써플라이),28,"트랜스,다이오드,칩 조립 전원공급장치(파워써플라이)"
105188,id_0999976,C,25,259,기타임가공,,,25,기타임가공
105189,id_0999986,C,10,106,벼를 가지고,벼를 도정하여,소비자에게 판매,10,벼를 가지고 벼를 도정하여 소비자에게 판매
105190,id_0999994,C,32,320,"카본열선,퀼팅구조체",조립.배열.봉제하여,의료용매트리스,32,"카본열선,퀼팅구조체 조립.배열.봉제하여 의료용매트리스"


In [135]:
df['text'] = df['text'].str.strip()

In [136]:
df['No.']=df.index

In [137]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,target,text,No.
0,id_0000006,C,29,291,철,절삭.용접,카프라배관자재,29,철 절삭.용접 카프라배관자재,0
1,id_0000008,C,10,107,쌀을 가지고,가공하여,떡제조,10,쌀을 가지고 가공하여 떡제조,1
2,id_0000011,C,10,102,멸치,"입고, 가공",,10,"멸치 입고, 가공",2
3,id_0000021,C,10,107,쌀을가지고,가루로 분쇄하여 떡을 쪄서,백설기 꿀떡 절편판매,10,쌀을가지고 가루로 분쇄하여 떡을 쪄서 백설기 꿀떡 절편판매,3
4,id_0000027,C,22,221,고무,절단가공,"가스켓,다이아후레임",22,"고무 절단가공 가스켓,다이아후레임",4
...,...,...,...,...,...,...,...,...,...,...
105187,id_0999971,C,28,281,"트랜스,다이오드,칩",조립,전원공급장치(파워써플라이),28,"트랜스,다이오드,칩 조립 전원공급장치(파워써플라이)",105187
105188,id_0999976,C,25,259,기타임가공,,,25,기타임가공,105188
105189,id_0999986,C,10,106,벼를 가지고,벼를 도정하여,소비자에게 판매,10,벼를 가지고 벼를 도정하여 소비자에게 판매,105189
105190,id_0999994,C,32,320,"카본열선,퀼팅구조체",조립.배열.봉제하여,의료용매트리스,32,"카본열선,퀼팅구조체 조립.배열.봉제하여 의료용매트리스",105190


In [138]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [139]:
def text_to_token(df:pd.DataFrame):
  input_ids = list()
  token_type_ids = list()
  attention_mask = list()

  for i in range(len(df)):
    temp = tokenizer.encode_plus(df['text'].loc[i], add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask = True, 
            return_token_type_ids=True,
            truncation=True)
    input_ids.append(temp['input_ids'])
    token_type_ids.append(temp['token_type_ids'])
    attention_mask.append(temp['attention_mask'])

  return input_ids, token_type_ids, attention_mask

df['input_ids'], df['token_type_ids'], df['attention_mask'] = text_to_token(df)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [140]:
df[['input_ids', 'token_type_ids', 'attention_mask']].head()

,input_ids,token_type_ids,attention_mask
0,"[2, 4473, 4067, 6515, 54, 7003, 7225, 4635, 77...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,"[2, 517, 6756, 7088, 768, 517, 5330, 5452, 781...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,"[2, 517, 6202, 7483, 3838, 46, 517, 5330, 5452...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,"[2, 517, 6756, 7088, 5330, 7321, 517, 5330, 60...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[2, 993, 6228, 4067, 5788, 5330, 5452, 754, 75...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [141]:
## GPU
device = torch.device("cuda:0")

In [142]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [143]:
my_dict = {}
for i in range(len(df['target'].unique())):
  my_dict[df['target'].unique()[i]] = i

In [144]:
my_dict

{10: 1,
 11: 19,
 12: 24,
 13: 17,
 14: 5,
 15: 16,
 16: 11,
 17: 18,
 18: 14,
 19: 23,
 20: 8,
 21: 22,
 22: 2,
 23: 15,
 24: 12,
 25: 3,
 26: 10,
 27: 21,
 28: 7,
 29: 0,
 30: 20,
 31: 13,
 32: 4,
 33: 6,
 34: 9}

In [145]:
def target_to_label(text:str):
  return my_dict[text]

df['target_label'] = df['target'].map(target_to_label)

In [146]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,target,text,No.,input_ids,token_type_ids,attention_mask,target_label
0,id_0000006,C,29,291,철,절삭.용접,카프라배관자재,29,철 절삭.용접 카프라배관자재,0,"[2, 4473, 4067, 6515, 54, 7003, 7225, 4635, 77...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
1,id_0000008,C,10,107,쌀을 가지고,가공하여,떡제조,10,쌀을 가지고 가공하여 떡제조,1,"[2, 517, 6756, 7088, 768, 517, 5330, 5452, 781...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1
2,id_0000011,C,10,102,멸치,"입고, 가공",,10,"멸치 입고, 가공",2,"[2, 517, 6202, 7483, 3838, 46, 517, 5330, 5452...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1
3,id_0000021,C,10,107,쌀을가지고,가루로 분쇄하여 떡을 쪄서,백설기 꿀떡 절편판매,10,쌀을가지고 가루로 분쇄하여 떡을 쪄서 백설기 꿀떡 절편판매,3,"[2, 517, 6756, 7088, 5330, 7321, 517, 5330, 60...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
4,id_0000027,C,22,221,고무,절단가공,"가스켓,다이아후레임",22,"고무 절단가공 가스켓,다이아후레임",4,"[2, 993, 6228, 4067, 5788, 5330, 5452, 754, 75...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105187,id_0999971,C,28,281,"트랜스,다이오드,칩",조립,전원공급장치(파워써플라이),28,"트랜스,다이오드,칩 조립 전원공급장치(파워써플라이)",105187,"[2, 4773, 6025, 6664, 46, 5782, 7096, 6964, 59...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",7
105188,id_0999976,C,25,259,기타임가공,,,25,기타임가공,105188,"[2, 1258, 7586, 5330, 5452, 3]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]",3
105189,id_0999986,C,10,106,벼를 가지고,벼를 도정하여,소비자에게 판매,10,벼를 가지고 벼를 도정하여 소비자에게 판매,105189,"[2, 517, 6353, 6116, 768, 517, 6353, 6116, 517...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1
105190,id_0999994,C,32,320,"카본열선,퀼팅구조체",조립.배열.봉제하여,의료용매트리스,32,"카본열선,퀼팅구조체 조립.배열.봉제하여 의료용매트리스",105190,"[2, 4635, 6383, 6940, 6559, 46, 0, 7681, 5501,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4


In [147]:
df[['target', 'target_label']].head()

,target,target_label
0,29,0
1,10,1
2,10,1
3,10,1
4,22,2


In [148]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,target,text,No.,input_ids,token_type_ids,attention_mask,target_label
0,id_0000006,C,29,291,철,절삭.용접,카프라배관자재,29,철 절삭.용접 카프라배관자재,0,"[2, 4473, 4067, 6515, 54, 7003, 7225, 4635, 77...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
1,id_0000008,C,10,107,쌀을 가지고,가공하여,떡제조,10,쌀을 가지고 가공하여 떡제조,1,"[2, 517, 6756, 7088, 768, 517, 5330, 5452, 781...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1
2,id_0000011,C,10,102,멸치,"입고, 가공",,10,"멸치 입고, 가공",2,"[2, 517, 6202, 7483, 3838, 46, 517, 5330, 5452...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1
3,id_0000021,C,10,107,쌀을가지고,가루로 분쇄하여 떡을 쪄서,백설기 꿀떡 절편판매,10,쌀을가지고 가루로 분쇄하여 떡을 쪄서 백설기 꿀떡 절편판매,3,"[2, 517, 6756, 7088, 5330, 7321, 517, 5330, 60...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
4,id_0000027,C,22,221,고무,절단가공,"가스켓,다이아후레임",22,"고무 절단가공 가스켓,다이아후레임",4,"[2, 993, 6228, 4067, 5788, 5330, 5452, 754, 75...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105187,id_0999971,C,28,281,"트랜스,다이오드,칩",조립,전원공급장치(파워써플라이),28,"트랜스,다이오드,칩 조립 전원공급장치(파워써플라이)",105187,"[2, 4773, 6025, 6664, 46, 5782, 7096, 6964, 59...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",7
105188,id_0999976,C,25,259,기타임가공,,,25,기타임가공,105188,"[2, 1258, 7586, 5330, 5452, 3]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]",3
105189,id_0999986,C,10,106,벼를 가지고,벼를 도정하여,소비자에게 판매,10,벼를 가지고 벼를 도정하여 소비자에게 판매,105189,"[2, 517, 6353, 6116, 768, 517, 6353, 6116, 517...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1
105190,id_0999994,C,32,320,"카본열선,퀼팅구조체",조립.배열.봉제하여,의료용매트리스,32,"카본열선,퀼팅구조체 조립.배열.봉제하여 의료용매트리스",105190,"[2, 4635, 6383, 6940, 6559, 46, 0, 7681, 5501,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4


In [149]:
# dataset_train = df[['No.', 'text', 'target_label']][:int(len(df)*0.8)]
# dataset_test = df[['No.', 'text', 'target_label']][int(len(df)*0.8):]
dataset_train, dataset_test = train_test_split(df[['No.', 'text', 'target_label']], test_size=0.2, random_state=42, shuffle=True)
dataset_train.to_csv('dataset_train.txt', index=False, header=True, sep='\t', encoding='utf8')
dataset_test.to_csv('dataset_test.txt', index=False, header=True, sep='\t', encoding='utf8')

In [150]:
dataset_train = nlp.data.TSVDataset('/content/dataset_train.txt', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/dataset_test.txt', field_indices=[1,2], num_discard_samples=1)

In [151]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [152]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [153]:
## Setting parameters
max_len = 256
batch_size = 8
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [154]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [155]:
next(iter(data_train))

(array([   2, 4213, 6241,  517,   46,  517,  441,  389,  391, 4213, 6241,
         517,   46, 1260, 5330, 5462, 6141,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [156]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [157]:
len(df['target_label'].unique())

25

In [158]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=520,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [159]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [160]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [161]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [162]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [163]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [164]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [165]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    # torch.cuda.empty_cache()

KeyboardInterrupt: ignored